In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

<br>

In [25]:
df = pd.read_csv("../fifa24_db/pdb_24.csv")

# Team Power Rating

In [39]:
def TPR(df, top_N_players=18):
    '''
    Team Power Rating
    '''
    # Group the DataFrame by club_name
    club_groups = df.groupby('club_name')

    # Initialize an empty list to store the DataFrames
    dfs = []

    # Iterate over each club
    for club_name, club_data in club_groups:
        # Sort the values by "overall" in descending order and select top 18 players
        top_players = club_data.sort_values(by='overall', ascending=False).head(top_N_players)

        # Separate players by positions
        goalkeepers = top_players[top_players['player_positions'].str.contains('GK')]
        outfield_players = top_players[~top_players['player_positions'].str.contains('GK')]

        # Calculate the mean of the attributes for outfield players
        mean_outfield_attributes = outfield_players.mean()

        # Create a DataFrame with club information and mean outfield attributes
        club_df = pd.DataFrame({
            "league_id": int(top_players.iloc[0]['league_id']),
            "club_team_id": int(top_players.iloc[0]['club_team_id']),  # Assuming club_team_id is the same for all players in a club
            "league_name": top_players.iloc[0]['league_name'],
            "club_name": club_name,
            "power": int(mean_outfield_attributes['overall']),  # Mean overall rating
        }, index=[0])  # Use index=[0] to ensure it's a DataFrame with a single row

        # Calculate the mean of the attributes for goalkeepers
        if not goalkeepers.empty:
            mean_goalkeeper_attributes = goalkeepers.mean()
            # Add mean goalkeeper attributes to the club DataFrame
            club_df["mean_goalkeeping_diving"] = int(mean_goalkeeper_attributes['goalkeeping_diving'])
            club_df["mean_goalkeeping_handling"] = int(mean_goalkeeper_attributes['goalkeeping_handling'])
            club_df["mean_goalkeeping_kicking"] = int(mean_goalkeeper_attributes['goalkeeping_kicking'])
            club_df["mean_goalkeeping_positioning"] = int(mean_goalkeeper_attributes['goalkeeping_positioning'])
            club_df["mean_goalkeeping_reflexes"] = int(mean_goalkeeper_attributes['goalkeeping_reflexes'])

        # Add mean attributes for outfield players
        outfield_attributes = [
            'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
            'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
            'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
            'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
            'movement_agility', 'movement_reactions', 'movement_balance',
            'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
            'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
            'mentality_positioning', 'mentality_vision', 'mentality_penalties',
            'mentality_composure', 'defending_marking_awareness',
            'defending_standing_tackle', 'defending_sliding_tackle'
        ]
        for attribute in outfield_attributes:
            club_df[f"mean_{attribute}"] = int(mean_outfield_attributes[attribute])

        # Append the DataFrame to the list
        dfs.append(club_df)

    # Concatenate all DataFrames in the list
    merged = pd.concat(dfs, ignore_index=True)

    # Display the resulting DataFrame
    return merged.sort_values("power", ascending=False)

In [43]:
x = TPR(df, 18).head(n=5)
x

,league_id,club_team_id,league_name,club_name,power,mean_goalkeeping_diving,mean_goalkeeping_handling,mean_goalkeeping_kicking,mean_goalkeeping_positioning,mean_goalkeeping_reflexes,...,mean_power_long_shots,mean_mentality_aggression,mean_mentality_interceptions,mean_mentality_positioning,mean_mentality_vision,mean_mentality_penalties,mean_mentality_composure,mean_defending_marking_awareness,mean_defending_standing_tackle,mean_defending_sliding_tackle
114,53,243,La Liga,Real Madrid,84,82.0,83.0,80.0,84.0,88.0,...,69,75,70,73,76,62,80,69,70,67
87,13,10,Premier League,Manchester City,84,81.0,77.0,88.0,82.0,85.0,...,67,79,71,69,75,61,82,72,74,67
44,19,21,Bundesliga,FC Bayern München,84,79.0,77.0,76.0,80.0,80.0,...,71,70,65,75,75,58,81,63,64,61
43,53,241,La Liga,FC Barcelona,83,86.0,85.0,89.0,86.0,91.0,...,63,73,71,70,75,57,79,67,71,66
106,16,73,Ligue 1,Paris Saint Germain,83,87.0,81.0,77.0,83.0,88.0,...,63,72,68,71,72,60,81,66,67,64


In [28]:
team_id = 10
top18=TPR(df, 18)[TPR(df, 18)["club_team_id"]==team_id].iloc[0]["power"]

In [37]:
x = TPR(df)
result = x[x['club_name'].str.contains("saler", case=False)]
result

,league_id,club_team_id,league_name,club_name,power,mean_goalkeeping_diving,mean_goalkeeping_handling,mean_goalkeeping_kicking,mean_goalkeeping_positioning,mean_goalkeeping_reflexes,...,mean_power_long_shots,mean_mentality_aggression,mean_mentality_interceptions,mean_mentality_positioning,mean_mentality_vision,mean_mentality_penalties,mean_mentality_composure,mean_defending_marking_awareness,mean_defending_standing_tackle,mean_defending_sliding_tackle
122,31,110373,Serie A,Salernitana,73,77.5,73.0,72.0,77.0,80.0,...,55.125,67.25,64.3125,58.8125,62.0625,53.0625,69.875,63.5625,64.6875,62.5


<br>

<br>

# Team Finishing Rating

In [30]:
def TFR(df, top_N_players=18):
    '''
    Team Finishing Rating
    '''
    # Group the DataFrame by club_name
    club_groups = df.groupby('club_name')

    # Initialize an empty list to store the DataFrames
    dfs = []

    # Iterate over each club
    for club_name, club_data in club_groups:
        # Sort the values by "overall" in descending order and select top 18 players
        top_players = club_data.sort_values(by='overall', ascending=False).head(top_N_players)
        
        # Separate players by positions
        attackers = top_players[top_players['player_positions'].str.contains('ST|CF|LW|RW|LF|RF')]
        midfielders = top_players[top_players['player_positions'].str.contains('CAM|LM|RM|CM|CDM')]
        defenders = top_players[top_players['player_positions'].str.contains('CB|LB|RB|LWB|RWB')]
        
        # Calculate mean attributes for each position group
        mean_attacking_finishing = attackers['attacking_finishing'].mean()
        mean_shot_power = midfielders['power_shot_power'].mean()
        mean_long_shots = midfielders['power_long_shots'].mean()
        mean_heading_accuracy = defenders['attacking_heading_accuracy'].mean()
        mean_power = top_players['overall'].mean()
        
        
        # Calculate the "finishing" rating based on positions
        finishing_rating = (mean_attacking_finishing * 0.4 +
                           mean_shot_power * 0.35 +
                           mean_long_shots * 0.2 +
                           mean_heading_accuracy * 0.05)

        # Create a DataFrame with club information and finisher rating
        club_df = pd.DataFrame({
            "league_id": top_players.iloc[0]['league_id'],
            "club_team_id": top_players.iloc[0]['club_team_id'],  
            "league_name": top_players.iloc[0]['league_name'],
            "club_name": club_name,
            "power":int(mean_power),
            "finishing": int(finishing_rating)
        }, index=[0])

        # Append the DataFrame to the list
        dfs.append(club_df)

    # Concatenate all DataFrames in the list
    merged = pd.concat(dfs, ignore_index=True)

    # Display the resulting DataFrame
    return merged.sort_values("finishing", ascending=False)

<br>

<br>

# Team Passing Rating

In [31]:
df.columns

Index(['player_id', 'short_name', 'long_name', 'player_positions', 'overall',
       'potential', 'value_eur', 'wage_eur', 'dob', 'height_cm',
       'club_team_id', 'club_name', 'league_id', 'league_name',
       'club_position', 'club_jersey_number', 'nationality_id',
       'nationality_name', 'preferred_foot', 'attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance',
       'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_stan

In [32]:
def TPSR(df, top_N_players=18):
    '''
    Team Passing Rating
    '''
    # Group the DataFrame by club_name
    club_groups = df.groupby('club_name')

    # Initialize an empty list to store the DataFrames
    dfs = []

    # Iterate over each club
    for club_name, club_data in club_groups:
        # Sort the values by "overall" in descending order and select top 18 players
        top_players = club_data.sort_values(by='overall', ascending=False).head(top_N_players)
        
        # Separate players by positions
        midfielders = top_players[top_players['player_positions'].str.contains('CAM|LM|RM|CM|CDM')]
        
        # Calculate mean attributes for midfielders
        mean_short_passing = midfielders['attacking_short_passing'].mean()
        mean_long_passing = midfielders['skill_long_passing'].mean()
        mean_curve = midfielders['skill_curve'].mean()
        mean_ball_control = midfielders['skill_ball_control'].mean()
        mean_vision = midfielders['mentality_vision'].mean()
        mean_power = top_players['overall'].mean()
        
        # Calculate the "passing" rating based on midfielders
        passing_rating = (mean_short_passing * 0.25 +
                          mean_long_passing * 0.2 +
                          mean_curve * 0.2 +
                          mean_ball_control * 0.2 +
                          mean_vision * 0.15)

        # Create a DataFrame with club information and passing rating
        club_df = pd.DataFrame({
            "league_id": top_players.iloc[0]['league_id'],
            "club_team_id": top_players.iloc[0]['club_team_id'],  
            "league_name": top_players.iloc[0]['league_name'],
            "club_name": club_name,
            "power": int(mean_power),
            "passing": int(passing_rating)
        }, index=[0])

        # Append the DataFrame to the list
        dfs.append(club_df)

    # Concatenate all DataFrames in the list
    merged = pd.concat(dfs, ignore_index=True)

    # Display the resulting DataFrame
    return merged.sort_values("passing", ascending=False)

# Example usage:
tpsr_data = TPSR(df)